In [53]:
import os 

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, Conv2D, MaxPooling2D, Dropout, LSTM, Bidirectional, Flatten 
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_curve



Using TensorFlow backend.


In [54]:
x_train = np.load(os.path.join('train_data_proc','x_0.npy'))
y_train = np.load(os.path.join('train_data_proc', 'y_0.npy'))

x_test = np.load(os.path.join('test_data_proc','x_0.npy'))
y_test = np.load(os.path.join('test_data_proc', 'y_0.npy'))

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(11500, 88)
(11500, 88)
(4800, 88)
(4800, 88)


In [55]:
context_window_size = 7
frequencies_size = x_train.shape[1]
sequence_size = 100
batch_size = 1

output_shape = y_train.shape[1]

In [79]:
keras.backend.clear_session()

In [80]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'), \
           input_shape=(sequence_size, context_window_size,frequencies_size,1)
    )
)
model.add(Dropout(0.5))
model.add(TimeDistributed(MaxPooling2D((1,3), strides=(1,1))))
          
model.add(TimeDistributed(Conv2D(128, (1,3), activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(MaxPooling2D((1,3), strides=(1,1))))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed(Dense(200)))
model.add(Dropout(0.5))

# model.add(Bidirectional(LSTM(128, return_sequences=False, dropout=0.5), merge_mode ='ave'))

model.add(TimeDistributed(Dense(output_shape, activation='sigmoid')))
model.compile(loss='categorical_crossentropy', optimizer='adam')
          


In [81]:
inp = model.input                                           # input placeholder
outputs = [(layer.input,layer.output) for layer in model.layers]          # all layer outputs
print (outputs)

[(<tf.Tensor 'time_distributed_1_input:0' shape=(?, 100, 7, 88, 1) dtype=float32>, <tf.Tensor 'time_distributed_1/Reshape_1:0' shape=(?, 100, 5, 86, 64) dtype=float32>), (<tf.Tensor 'time_distributed_1/Reshape_1:0' shape=(?, 100, 5, 86, 64) dtype=float32>, <tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 100, 5, 86, 64) dtype=float32>), (<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 100, 5, 86, 64) dtype=float32>, <tf.Tensor 'time_distributed_2/Reshape_1:0' shape=(?, 100, 5, 84, 64) dtype=float32>), (<tf.Tensor 'time_distributed_2/Reshape_1:0' shape=(?, 100, 5, 84, 64) dtype=float32>, <tf.Tensor 'time_distributed_3/Reshape_1:0' shape=(?, 100, 5, 82, 128) dtype=float32>), (<tf.Tensor 'time_distributed_3/Reshape_1:0' shape=(?, 100, 5, 82, 128) dtype=float32>, <tf.Tensor 'dropout_2/cond/Merge:0' shape=(?, 100, 5, 82, 128) dtype=float32>), (<tf.Tensor 'dropout_2/cond/Merge:0' shape=(?, 100, 5, 82, 128) dtype=float32>, <tf.Tensor 'time_distributed_4/Reshape_1:0' shape=(?, 100, 5, 80, 128) dtyp

In [82]:
import datagen

In [87]:
import importlib
datagen = importlib.reload(datagen)

data_generator = datagen.DataGenerator('./train_data_proc/', 100,256, 7, 'x_[0-9]+.npy', 'y_[0-9]+.npy')

now listing inputs
['x_0.npy']
now listing outputs
['y_0.npy']


In [88]:
print(data_generator)


In [89]:
model.fit_generator(generator=data_generator.__getitem__(),
                    use_multiprocessing=True,
                    workers=6, epochs=5, steps_per_epoch=1)

Epoch 1/5
y_0.npy
y_0.npy
y_0.npy
y_0.npy
y_0.npy
11500
Epoch 1/5
y_0.npy
11500
11500
11500
3
3
11500
3
3
3
11500
3


ValueError: Error when checking input: expected time_distributed_1_input to have 5 dimensions, but got array with shape (100, 7, 88, 1)